refer to 
* https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb#scrollTo=dsBo6RCtQmwx

In [1]:
import pandas as pd
import tensorflow as tf

2022-06-29 11:15:37.341323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-06-29 11:15:37.341369: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from tensorflow import keras
import os
import re

In [3]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df


In [4]:
train, test = download_and_load_datasets()

In [5]:
train.shape

(25000, 3)

In [6]:
train.head()

,sentence,sentiment,polarity
0,The Invisible Man is a fantastic movie from 19...,2,0
1,Steve Carrel Proves himself to be a great lead...,10,1
2,This is a very exciting and romantic film. I h...,10,1
3,This superb film draws on a variety of talente...,9,1
4,I do not believe all the praise for this movie...,3,0


In [7]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [8]:
import tokenization

In [49]:
tokenizer = tokenization.FullTokenizer(
      vocab_file="../vocab.txt")  # 中文BERT以字为最终的token

In [50]:
tokenizer.tokenize('woshssDfsfd')

['w', '##os', '##hs', '##sd', '##fs', '##f', '##d']

In [51]:
tokenizer.tokenize('woshssdfsfd')

['w', '##os', '##hs', '##sd', '##fs', '##f', '##d']

In [52]:
tokenizer.tokenize('大美边疆')

['大', '美', '边', '疆']

In [54]:
tokenizer.tokenize('哎呦喂hellod')

['哎', '呦', '喂', 'hello', '##d']